In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import fake_useragent
import re
import warnings
warnings.filterwarnings("ignore")

ua = fake_useragent.UserAgent()

In [2]:
def get_url(tot_pages):
    for page in range(51,tot_pages+1, 51):
        if page % log_frequency == 0:
            print(f'Log: working over page {page}...')       
        
        url_main = f'https://beg-dorozhki.ru/category/offset{page}'
        response = requests.get(url_main, headers = {'user-agent': ua.random}, verify=False)
        soup = BeautifulSoup(response.content, 'lxml')
        data = soup.find_all(class_ = 'ui-link')
        for item in data:
            if bool(re.search(pattern = re.escape('/product/'), string = item['href'])):
                item_url = 'https://beg-dorozhki.ru' + item['href']
                yield item_url

In [3]:
def get_item(category, tot_pages):
    count = 0
    for url_item in get_url(tot_pages):
        count += 1
        response = requests.get(url_item, headers = {'user-agent': ua.random}, verify=False)
        soup = BeautifulSoup(response.content, 'lxml')

        try:
            title = soup.find('h1').text
            title = title.replace('\r', '').replace('\t', '').replace('\n', '').strip()
        except:
            title = ''

        try:
            article = soup.find(class_ = 'b-goods__article').text
            article = article.replace(' ', '').replace('\n', '')
            article = article.replace('Артикул:', '').strip()
        except:
            article = ''

        try:
            image = soup.find(class_ = 'img-scale', src = True)
            url = 'https://beg-dorozhki.ru' + image['src']
            image_ref = f"images/{category}/{count}_begdorozhki_{article}.jpeg"
            with open(f'images/{category}/{count}_begdorozhki_{article}.jpeg', 'wb') as f:
                r = requests.get(url)
                f.write(r.content)     

        except:
            print(f'Log: no image for {article} found...')
            url = ''
            image_ref = ''

        try:
            char = soup.find_all(class_ = 'b-goods-specif__item param')
            char_string = ''
            for ch in char:
                children = ch.children
                for child in children:
                    char_string += child.text
                    char_string += ' '
            chars = char_string.strip()
        except:
            chars = ''

        try:
            desc = soup.find(class_ = 'b-goods-info')
            desc = desc.text.replace('\n', ' ').strip()
        except:
            desc = ''

        try:
            price = soup.find(class_ = 'b-goods-price__new').text
            price = price.replace(' ', '').replace('руб.', '')
        except:
            price =''

        try:
            cat = soup.find(class_ = 'breadcrumb').text
            cat_lst = []
            for i in cat.split('\n'):
                if i != '':
                    cat_lst.append(i)
        except:
            cat_lst = []
        try:
            cat_1 = cat_lst[0].strip()
        except:
            cat_1 = ''
        try:
            cat_2 = cat_lst[1].strip()
        except:
            cat_2 = ''
        try:
            cat_3 = cat_lst[2].strip()
        except:
            cat_3 = ''

        yield  title, article, price, cat_1, cat_2, cat_3, url, desc, chars, image_ref

In [4]:
df_columns = ['title', 'article','price', 'cat_1', 'cat_2', 'cat_3','url', 
              'description', 'caracteristics', 'img_ref']

df = pd.DataFrame(columns = df_columns)
save_frequency = 100
log_frequency = 300

In [5]:
import time

def to_csv(category, tot_pages):
    counter = 0  

    for item in get_item(category, tot_pages):
            data = []
            for i in range(len(df_columns)):
                data.append(item[i])
            df.loc[len(df.index)] = data
            time.sleep(0.05)

            counter += 1
            if counter % save_frequency == 0:
                df.to_csv(f'{category}beg_dorozh.csv')
                print(f'{counter} items saved in csv..')
    df.to_csv(f'{category}beg_dorozh.csv')

    print(f'\nTotal items saved: {counter}')  

In [6]:
to_csv('begovye_dorozhki', 663)

100 items saved in csv..
200 items saved in csv..
300 items saved in csv..
400 items saved in csv..
500 items saved in csv..
600 items saved in csv..

Total items saved: 663
